In [1]:
# %matplotlib widget

import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.fits")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.fits")
out_arc_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{lamps}.fits")

# overwrite or not
OVERWRITE = True

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
CHANNEL_WL = {"b": (3600, 5930), "r": (5660, 7720), "z": (7470, 9800)}
LAMPS = "neon_xenon"
REF_FIBER = 319

masters_arc = get_masters_metadata(path_pattern=out_arc_path, mjd="super", kind="xarc", lamps=LAMPS).sort_values("camera")
print(masters_arc.to_string())

for _, marc in masters_arc.iterrows():
    print(marc)

    try:
        rssMethod.detWaveSolution_drp(
            in_arc=marc.path,
            out_wave=out_arc_path.format(mjd=marc.mjd, kind="wave", camera=marc.camera, lamps=marc.lamps),
            out_lsf=out_arc_path.format(mjd=marc.mjd, kind="lsf", camera=marc.camera, lamps=marc.lamps),
            in_ref_lines=f"../../python/lvmdrp/etc/lvm-{LAMPS}_nist_{marc.camera[0]}1.txt",
            ref_fiber=REF_FIBER, poly_disp=3, poly_fwhm=3, poly_cros=3, poly_kinds="poly,poly,poly",
            flux_min="10.0", fwhm_max="5.0", rel_flux_limits="0.001,100", aperture=10, plot=2
        )
        rssMethod.createPixTable_drp(
            in_rss=out_arc_path.format(mjd=marc.mjd, kind="xarc", camera=marc.camera, lamps=marc.lamps),
            out_rss=out_arc_path.format(mjd=marc.mjd, kind="warc", camera=marc.camera, lamps=marc.lamps),
            arc_wave=out_arc_path.format(mjd=marc.mjd, kind="wave", camera=marc.camera, lamps=marc.lamps),
            arc_fwhm=out_arc_path.format(mjd=marc.mjd, kind="lsf", camera=marc.camera, lamps=marc.lamps)
        )
        wave_range = CHANNEL_WL[marc.camera[0]]
        rssMethod.resampleWave_drp(
            in_rss=out_arc_path.format(mjd=marc.mjd, kind="warc", camera=marc.camera, lamps=marc.lamps),
            out_rss=out_arc_path.format(mjd=marc.mjd, kind="harc", camera=marc.camera, lamps=marc.lamps),
            start_wave=wave_range[0], end_wave=wave_range[1], disp_pix=1.0, method="linear",
            err_sim=10, parallel="auto", extrapolate=True
        )
    except Exception as e:
        rssMethod.rss_logger.error(e)

     mjd  kind camera       lamps                                      path
7  super  xarc     b1  neon_xenon  ./data/lvm-super-xarc-b1-neon_xenon.fits
8  super  xarc     b2  neon_xenon  ./data/lvm-super-xarc-b2-neon_xenon.fits
5  super  xarc     b3  neon_xenon  ./data/lvm-super-xarc-b3-neon_xenon.fits
0  super  xarc     r1  neon_xenon  ./data/lvm-super-xarc-r1-neon_xenon.fits
3  super  xarc     r2  neon_xenon  ./data/lvm-super-xarc-r2-neon_xenon.fits
2  super  xarc     r3  neon_xenon  ./data/lvm-super-xarc-r3-neon_xenon.fits
4  super  xarc     z1  neon_xenon  ./data/lvm-super-xarc-z1-neon_xenon.fits
1  super  xarc     z2  neon_xenon  ./data/lvm-super-xarc-z2-neon_xenon.fits
6  super  xarc     z3  neon_xenon  ./data/lvm-super-xarc-z3-neon_xenon.fits
mjd                                          super
kind                                          xarc
camera                                          b1
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-b1-n

measuring arc lines upwards from ref_fiber = 319:   1%|3                                               | 2/319 [00:00<00:48,  6.48fiber/s]

[WARNING]: Number of calls to function has reached maxfev = 9999. (RuntimeWarning)


measuring arc lines upwards from ref_fiber = 319: 100%|##############################################| 319/319 [01:19<00:00,  4.00fiber/s]
measuring arc lines downwards from ref_fiber = 319: 100%|############################################| 318/318 [01:21<00:00,  3.90fiber/s]


[INFO]: smoothing FWHM of guess lines along cross-dispersion axis using 3-deg polynomials
[INFO]: fitting wavelength solutions using 3-deg polynomials
[INFO]: finished wavelength fitting with median RMS = 3.0747 AA (5.24089 pix)
[INFO]: fitting LSF solutions using 3-deg polynomials
[INFO]: finished LSF fitting with median RMS = 0.404801 AA (0.698271 pix)
[INFO]: updating header and writing wavelength/LSF to './data/lvm-super-wave-b1-neon_xenon.fits' and './data/lvm-super-lsf-b1-neon_xenon.fits'
mjd                                          super
kind                                          xarc
camera                                          b2
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-b2-neon_xenon.fits
Name: 8, dtype: object
[INFO]: reading guess lines from '../../python/lvmdrp/etc/lvm-neon_xenon_nist_b1.txt'
[INFO]: going to use fiber 319 as reference
[INFO]: number of guess lines in file 25 percentage masked  4 %
[INFO]: going to use 24 gues

...

[INFO]: smoothing FWHM of guess lines along cross-dispersion axis using 3-deg polynomials
[ERROR]: Coefficient array is empty
mjd                                          super
kind                                          xarc
camera                                          z2
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-z2-neon_xenon.fits
Name: 1, dtype: object
[INFO]: reading guess lines from '../../python/lvmdrp/etc/lvm-neon_xenon_nist_z1.txt'
[INFO]: going to use fiber 319 as reference
[INFO]: number of guess lines in file 47 percentage masked  10.6383 %
[INFO]: going to use 42 guess lines
[INFO]: reading arc from './data/lvm-super-xarc-z2-neon_xenon.fits'
[INFO]: calculating shift in guess lines using CC
[INFO]: maximum CC shift = 9 pix
[INFO]: measuring arc lines for each fiber from reference fiber 319, flux_min = 10.0, fwhm_max = 5.0 and relative flux limits [0.001, 100.0]


measuring arc lines upwards from ref_fiber = 319: 100%|##############################################| 319/319 [00:40<00:00,  7.96fiber/s]
measuring arc lines downwards from ref_fiber = 319: 100%|############################################| 318/318 [00:38<00:00,  8.31fiber/s]


[INFO]: smoothing FWHM of guess lines along cross-dispersion axis using 3-deg polynomials
[INFO]: fitting wavelength solutions using 3-deg polynomials
[INFO]: finished wavelength fitting with median RMS = 1.07099 AA (1.78765 pix)
[INFO]: fitting LSF solutions using 3-deg polynomials
[INFO]: finished LSF fitting with median RMS = 0.286593 AA (0.477416 pix)
[INFO]: updating header and writing wavelength/LSF to './data/lvm-super-wave-z2-neon_xenon.fits' and './data/lvm-super-lsf-z2-neon_xenon.fits'
mjd                                          super
kind                                          xarc
camera                                          z3
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-z3-neon_xenon.fits
Name: 6, dtype: object
[INFO]: reading guess lines from '../../python/lvmdrp/etc/lvm-neon_xenon_nist_z1.txt'
[INFO]: going to use fiber 319 as reference
[INFO]: number of guess lines in file 47 percentage masked  10.6383 %
[INFO]: going to use 

measuring arc lines upwards from ref_fiber = 319: 100%|##############################################| 319/319 [00:37<00:00,  8.58fiber/s]
measuring arc lines downwards from ref_fiber = 319: 100%|############################################| 318/318 [00:48<00:00,  6.57fiber/s]


[INFO]: smoothing FWHM of guess lines along cross-dispersion axis using 3-deg polynomials
[INFO]: fitting wavelength solutions using 3-deg polynomials
[INFO]: finished wavelength fitting with median RMS = 2.42319 AA (4.06676 pix)
[INFO]: fitting LSF solutions using 3-deg polynomials
[INFO]: finished LSF fitting with median RMS = 0.426792 AA (0.703787 pix)
[INFO]: updating header and writing wavelength/LSF to './data/lvm-super-wave-z3-neon_xenon.fits' and './data/lvm-super-lsf-z3-neon_xenon.fits'


In [5]:
fiberflats = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="xfiberflat", exptime="x").sort_values("camera")

for _, fiberflat in fiberflats.iterrows():
    print(fiberflat)
    
    rssMethod.createPixTable_drp(
        in_rss=fiberflat.path,
        out_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="wfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        arc_wave=out_arc_path.format(mjd=fiberflat.mjd, kind="wave", camera=fiberflat.camera, lamps=LAMPS),
        arc_fwhm=out_arc_path.format(mjd=fiberflat.mjd, kind="lsf", camera=fiberflat.camera, lamps=LAMPS)
    )

    wave_range = CHANNEL_WL[fiberflat.camera[0]]
    rssMethod.resampleWave_drp(
        in_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="wfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        out_rss=out_calib_path.format(mjd=fiberflat.mjd, kind="hfiberflat", camera=fiberflat.camera, exptime=fiberflat.exptime),
        start_wave=wave_range[0], end_wave=wave_range[1], disp_pix=1.0, method="linear",
        err_sim=10, parallel="auto", extrapolate=True
    )

mjd                                        super
kind                                  xfiberflat
camera                                        b1
exptime                                        x
path       ./data/lvm-super-xfiberflat-b1-x.fits
Name: 3, dtype: object
mjd                                        super
kind                                  xfiberflat
camera                                        b2
exptime                                        x
path       ./data/lvm-super-xfiberflat-b2-x.fits
Name: 7, dtype: object
mjd                                        super
kind                                  xfiberflat
camera                                        b3
exptime                                        x
path       ./data/lvm-super-xfiberflat-b3-x.fits
Name: 4, dtype: object
mjd                                        super
kind                                  xfiberflat
camera                                        r1
exptime                                        x


In [6]:
arcs = get_masters_metadata(path_pattern=out_arc_path, mjd="super", kind="xarc", lamps=LAMPS).sort_values("camera")

for _, arc in arcs.iterrows():
    print(arc)
    
    rssMethod.createPixTable_drp(
        in_rss=arc.path,
        out_rss=out_arc_path.format(mjd=arc.mjd, kind="warc", camera=arc.camera, lamps=LAMPS),
        arc_wave=out_arc_path.format(mjd=arc.mjd, kind="wave", camera=arc.camera, lamps=LAMPS),
        arc_fwhm=out_arc_path.format(mjd=arc.mjd, kind="lsf", camera=arc.camera, lamps=LAMPS)
    )

    wave_range = CHANNEL_WL[arc.camera[0]]
    rssMethod.resampleWave_drp(
        in_rss=out_arc_path.format(mjd=arc.mjd, kind="warc", camera=arc.camera, lamps=LAMPS),
        out_rss=out_arc_path.format(mjd=arc.mjd, kind="harc", camera=arc.camera, lamps=LAMPS),
        start_wave=wave_range[0], end_wave=wave_range[1], disp_pix=1.0, method="linear",
        err_sim=10, parallel="auto", extrapolate=True
    )

mjd                                          super
kind                                          xarc
camera                                          b1
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-b1-neon_xenon.fits
Name: 7, dtype: object
mjd                                          super
kind                                          xarc
camera                                          b2
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-b2-neon_xenon.fits
Name: 8, dtype: object
mjd                                          super
kind                                          xarc
camera                                          b3
lamps                                   neon_xenon
path      ./data/lvm-super-xarc-b3-neon_xenon.fits
Name: 5, dtype: object
mjd                                          super
kind                                          xarc
camera                                          r1
lamps        